In [ ]:
!pip install emoji
!pip install pythainlp
!pip install stop_words
!pip install deepcut
!pip install attacut

     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42175 sha256=c8c4a9e333a07707aec71a614850a5c1b1343bd602f472cfe4b0207918f56fd0
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji
     |████████████████████████████████| 11.0MB 36.5MB/s 
     |████████████████████████████████| 276kB 45.2MB/s 
  Created wheel for marisa-trie: filename=marisa_trie-0.7.5-cp36-cp36m-linux_x86_64.whl size=862199 sha256=4f0c17d41066335c31ee1934409df335b0b61ce3cfd306ee2f29888fafd63cb8
  Stored in directory: /root/.cache/pip/wheels/45/24/79/022624fc914f0e559fe8a1141aaff1f9df810905a13fc75d57
Successfully built marisa-trie
  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32916 sha256=222a92bec4dbdc59a5135e15f23f36a5ccacd121cb639a1b1ffc199dcda7bfd4
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3

     |████████████████████████████████| 1.3MB 2.8MB/s 
     |████████████████████████████████| 481kB 55.7MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
     |████████████████████████████████| 266kB 52.5MB/s 
     |████████████████████████████████| 757kB 16.1MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=496e5bac6b34933617fcdbae84e46e86897dfd2f8f4e7583af3dafe7714a33ac
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44104 sha256=66997461eabf690dd65c3a4f69a9ec8e5faabbaf839bb978f7427099cc2124b2
  Stored in directory: /root/.cache/pip/wheels/d9/45/dd/65f0b38450c47cf7e5312883deb97d065e030c5cca0a365030
Successfully built fire pyyaml
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.

In [ ]:
from google.colab import drive
import string
import re
import emoji
import pythainlp
from pythainlp.tokenize import dict_trie, word_tokenize
from pythainlp.corpus import thai_stopwords, thai_words
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words
import nltk
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
% matplotlib inline
import matplotlib.font_manager as fm
import matplotlib


[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!ls '/content/gdrive/My Drive/Colab Notebooks/Data Project/data/fix wrong type'

'fix wrong type.gsheet'   test_data.csv       train_data.csv
 test_data_01.csv	  train_data_01.csv  'train_data v2.csv'


**1. Import Data**

In [ ]:
import pandas as pd
import csv
path = '/content/gdrive/My Drive/Colab Notebooks/Data Project/data/fix wrong type/'
filename = 'test_data_01.csv'
labels, texts, rating_start, comment_time, shopid, itemid = [], [], [], [], [], []
with open(path+filename, 'r', newline='') as f:
  reader = csv.reader(f)
  header = next(reader)
  for row in reader:
    #print(row[2].strip())
    labels.append(row[6])
    texts.append(row[7].strip())
    rating_start.append(row[25])
    comment_time.append(row[8])
    shopid.append(row[26])
    itemid.append(row[16])
test_data = pd.DataFrame()
test_data['text'] = texts
test_data['label'] = labels
test_data['rating_start'] = rating_start
test_data['comment_time'] = comment_time
test_data['shopid'] = shopid
test_data['itemid'] = itemid

In [ ]:
test_data.head(20)

,text,label,rating_start,comment_time,shopid,itemid
0,เพิ่งลองสั่งครั้งแรกค่ะยังไม่เคยลองใช้ ทางร้าน...,0,5,9/2/2019 18:00,4343306,54485355
1,สินค้าดี มีคุณภาพ ..สินค้าดี มีคุณภาพ ..สินค้า...,0,5,10/4/2019 14:54,4343306,54485355
2,ห่อดีมาก พันbubble แน่นมากจนอยากจะหักดาว 55555...,0,5,3/30/2019 14:55,4343306,54485355
3,Estee Lauder Advanced Night Repair Synchroniz...,0,5,7/30/2019 12:28,4343306,54485355
4,คุณภาพของสินค้าดีมาก ความคุ้มค่าดีมาก ความรวดเ...,0,5,11/9/2018 6:40,4343306,54485355
5,ของแท้แน่นอนค่ะ ทางร้านบริการดีมาก มีของแถมให้...,0,5,3/4/2019 10:31,4343306,54485355
6,pack ของดีมาก มีถ่ายรูปแจ้งก่อนส่งสินค้า ส่งไว...,0,5,8/5/2019 12:43,4343306,54485355
7,คุณภาพของสินค้าดีมาก ความคุ้มค่าดีมาก ความรวดเ...,0,5,11/9/2018 12:18,4343306,54485355
8,ทางร้านจัดส่งสินค้ารวดเร็วดี สินค้าคุณภาพดี ch...,0,5,5/9/2019 13:41,4343306,54485355
9,สินค้าส่งเร็วค่ะ ทางร้านแจ้งตลอด เข้าไปอ่านจาก...,0,4,6/1/2019 1:43,4343306,54485355


In [ ]:
path = '/content/gdrive/My Drive/Colab Notebooks/Data Project/data/fix wrong type/'
filename = 'train_data_01.csv'
labels, texts, rating_start, comment_time, shopid, itemid = [], [], [], [], [], []
with open(path+filename, 'r', newline='') as f:
  reader = csv.reader(f)
  header = next(reader)
  for row in reader:
    #print(row[2].strip())
    labels.append(row[6])
    texts.append(row[7].strip())
    rating_start.append(row[25])
    comment_time.append(row[8])
    shopid.append(row[26])
    itemid.append(row[16])
train_data = pd.DataFrame()
train_data['text'] = texts
train_data['label'] = labels
train_data['rating_start'] = rating_start
train_data['comment_time'] = comment_time
train_data['shopid'] = shopid
train_data['itemid'] = itemid

In [ ]:
train_data.head(20)

,text,label,rating_start,comment_time,shopid,itemid
0,สีและเนื้อผลิตภัณฑ์ไม่เหมือนที่ใช้อยู่ค่ะ ตัวท...,1,1,11/13/2018 20:53,84219986,1592175112
1,คุ้มค่ากับการรอคอยแท้💯%,0,5,12/7/2018 21:19,84219986,1592175112
2,คุณภาพของสินค้าดี ความคุ้มค่าดีมาก ความรวดเร็ว...,0,5,8/2/2019 20:15,84219986,1592175112
3,คุณภาพของสินค้าดีมาก ความคุ้มค่าดีมาก ความรวดเ...,0,5,12/3/2018 19:21,84219986,1592175112
4,ตัวสินค้าต่างจากสินค้าจาก shop (สีจางกว่า กลิ่...,1,1,12/14/2018 23:23,84219986,1592175112
5,ส่งของเร็วคะ check code ขึ้น test กลิ่นผ่าน...,0,5,10/10/2019 14:27,84219986,1592175112
6,สินค้าpack มาดีมากค่ะ จัดส่งรวดเร็วทันใจ คุณภา...,0,5,5/4/2019 7:25,84219986,1592175112
7,ความคุ้มค่าดีมาก ความรวดเร็วในการจัดส่งดีมาก ก...,0,5,11/4/2018 15:59,84219986,1592175112
8,แต่จุกยางเก่านิดหนึ่ง แต่ขอลงแท้แน่นอนค่ะ,0,5,12/24/2018 12:46,84219986,1592175112
9,ส่งเร็ว pack ดี ไม่มีตำหนิ ของแท้ ป้ายคิงพาวเว...,0,5,10/4/2019 21:46,84219986,1592175112


**2. Clean Text**

In [ ]:
def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

In [ ]:
def clean_text(text):    
  text = text.replace(u'\xa0', u' ')
  text = text.replace(u'\u200b', u' ') 
  text = text.replace(u'\U0001f970', u' ') 
  text = text.replace(u'\U0001f9f4', u' ') 
  text = text.replace(u'\U0001f97a', u' ')
  text = remove_emoji(text)
  
  # ลบ เครื่องหมายคำพูด (punctuation) !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~
  for c in string.punctuation:
    text = re.sub(r'\{}'.format(c),'',text)
  
  # delete ‘’‼
  for c in '‘’‼“”ๆ…️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️️': 
    text = re.sub(r'\{}'.format(c),'',text)
    
  # ลบ separator เช่น \n \t
  #msg = ' '.join(msg.split())
    
  return text

In [ ]:
train_data['text'] = train_data['text'].apply(clean_text)
test_data['text'] = test_data['text'].apply(clean_text)

**3. Tokenize**

In [ ]:
nltk.download('words')
th_stop = tuple(thai_stopwords())
en_stop = tuple(get_stop_words('en'))
p_stemmer = PorterStemmer()

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
# custom_vocab = [
#     "สก้อตเทป",
#     "แพค",
#     "เซรั่ม",
#     "สิ้นค้า",
#     "คอนวี่",
#     "เคาเตอร์",
#     "ทรีทเม้นท์",
#     "แฟลตเซล",
#     "สิสค้า",
#     "เคาร์เตอร์",
#     "เทส",
#     "หนีด",
#     "อีฟแอนด์บอย",
#     "รับของ",
#     "ไม่รุ้",
#     "เคาท์เตอร์",
#     "เคาว์เตอร์",
#     "เคาทเตอร์",
#     "เค้าเตอร์"
# ]
# trie = dict_trie(dict_source=list(thai_words()) + custom_vocab)
stop = [word for word in thai_stopwords()]
stop.remove('ส่ง')
stop.append('ล่ะ')
stop.append('อะ')
stop.append('อะนะ')

In [ ]:
def split_word(text,engine='newmm'):
    
    tokens = word_tokenize(text, engine=engine, keep_whitespace=False)
    
    # Remove stop words ภาษาไทย และภาษาอังกฤษ
    tokens = [i for i in tokens if not i in stop]

    tokens = [i.lower() for i in tokens]
    
    # หารากศัพท์ภาษาไทย และภาษาอังกฤษ
    # English
    #tokens = [p_stemmer.stem(i) for i in tokens]
    
    # Thai
    # tokens_temp=[]
    # for i in tokens:
    #     w_syn = wordnet.synsets(i)
    #     if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
    #         tokens_temp.append(w_syn[0].lemma_names('tha')[0])
    #     else:
    #         tokens_temp.append(i)
    
    # tokens = tokens_temp
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]

    return tokens

In [ ]:
def split_word_newmm(text):
    
    tokens = word_tokenize(text, engine='newmm', keep_whitespace=False)
    
    # Remove stop words ภาษาไทย และภาษาอังกฤษ
    tokens = [i for i in tokens if not i in stop]

    tokens = [i.lower() for i in tokens]
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]

    return tokens

In [ ]:
def split_word_deepcut(text):
    
    tokens = word_tokenize(text, engine='deepcut', keep_whitespace=False)
    
    # Remove stop words ภาษาไทย และภาษาอังกฤษ
    tokens = [i for i in tokens if not i in stop]

    tokens = [i.lower() for i in tokens]
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]

    return tokens

In [ ]:
def split_word_attacut(text):
    
    tokens = word_tokenize(text, engine='attacut', keep_whitespace=False)
    
    # Remove stop words ภาษาไทย และภาษาอังกฤษ
    tokens = [i for i in tokens if not i in stop]

    tokens = [i.lower() for i in tokens]
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]

    return tokens

In [ ]:
# train_data['newmm'] = train_data['text'].apply(split_word)
# train_data['deepcut'] = train_data['text'].apply(split_word,engine='deepcut')
# train_data['attacut'] = train_data['text'].apply(split_word,engine='attacut')
# test_data['newmm'] = test_data['text'].apply(split_word)
# test_data['deepcut'] = test_data['text'].apply(split_word,engine='deepcut')
# test_data['attacut'] = test_data['text'].apply(split_word,engine='attacut')
# train_df['newmm'] = train_df['text'].apply(word_tokenize, keep_whitespace=False, custom_dict=trie)
# train_df['deepcut'] = train_df['text'].apply(word_tokenize,engine='deepcut', keep_whitespace=False)
# train_df['attacut'] = train_df['text'].apply(word_tokenize,engine='attacut', keep_whitespace=False)

In [ ]:
# for text, label, newmm in review_df.values:
#   print(text)
#   print(newmm, end='\n\n')
  #print(fix_wrong_type(newmm, correct_string), end='\n\n')
  #print(deepcut)
  #print(fix_wrong_type(deepcut, correct_string))
  #print(attacut)
  #print(fix_wrong_type(attacut, correct_string), end='\n\n')

In [ ]:
X_train = train_data['text']
X_test = test_data['text']
y_train = train_data['label']
y_test = test_data['label']

**Feature Engineering**

**CountVectorizer as features**

newmm

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Initialize a CountVectorizer object: count_vectorizer
count_vectorizer_newmm = CountVectorizer(analyzer=split_word_newmm)

# Transform the training data using only the 'text' column values: count_train 
count_train_newmm = count_vectorizer_newmm.fit_transform(X_train.values)

# Transform the test data using only the 'text' column values: count_test 
count_test_newmm = count_vectorizer_newmm.transform(X_test.values)

# Print the first 10 features of the count_vectorizer
print(count_vectorizer_newmm.get_feature_names()[:10])

['advance', 'advanced', 'age', 'air', 'amazing', 'and', 'anr', 'ant', 'app', 'april']


deepcut

In [ ]:
# Initialize a CountVectorizer object: count_vectorizer
count_vectorizer_deepcut = CountVectorizer(analyzer=split_word_deepcut)

# Transform the training data using only the 'text' column values: count_train 
count_train_deepcut = count_vectorizer_deepcut.fit_transform(X_train.values)

# Transform the test data using only the 'text' column values: count_test 
count_test_deepcut = count_vectorizer_deepcut.transform(X_test.values)

# Print the first 10 features of the count_vectorizer
print(count_vectorizer_deepcut.get_feature_names()[:10])

['1ดาว', '30ml', '50ml', '5ดาว', 'advance', 'advance night', 'advanced', 'age', 'air', 'amazing']


attacut

In [ ]:
# Initialize a CountVectorizer object: count_vectorizer
count_vectorizer_attacut = CountVectorizer(analyzer=split_word_attacut)

# Transform the training data using only the 'text' column values: count_train 
count_train_attacut = count_vectorizer_attacut.fit_transform(X_train.values)
/
# Transform the test data using only the 'text' column values: count_test 
count_test_attacut = count_vectorizer_attacut.transform(X_test.values)

# Print the first 10 features of the count_vectorizer
print(count_vectorizer_attacut.get_feature_names()[:10])

['advance', 'advanced', 'age', 'air', 'amazing', 'and', 'anr', 'ant', 'app', 'april']


**TF-IDF Vectors as features**

newmm

In [ ]:
# Initialize a TfidfVectorizer object: tfidf_vectorizer
tfidf_vectorizer_newmm = TfidfVectorizer(analyzer=split_word_newmm, max_df=0.7)

# Transform the training data: tfidf_train 
tfidf_train_newmm = tfidf_vectorizer_newmm.fit_transform(X_train.values)

# Transform the test data: tfidf_test 
tfidf_test_newmm = tfidf_vectorizer_newmm.transform(X_test.values)

# Print the first 10 features
print(tfidf_vectorizer_newmm.get_feature_names()[:10])

# Print the first 5 vectors of the tfidf training data
print(tfidf_train_newmm.A[:5])

['advance', 'advanced', 'age', 'air', 'amazing', 'and', 'anr', 'ant', 'app', 'april']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


deepcut

In [ ]:
# Initialize a TfidfVectorizer object: tfidf_vectorizer
tfidf_vectorizer_deepcut = TfidfVectorizer(analyzer=split_word_deepcut, max_df=0.7)

# Transform the training data: tfidf_train 
tfidf_train_deepcut = tfidf_vectorizer_deepcut.fit_transform(X_train.values)

# Transform the test data: tfidf_test 
tfidf_test_deepcut = tfidf_vectorizer_deepcut.transform(X_test.values)

# Print the first 10 features
print(tfidf_vectorizer_deepcut.get_feature_names()[:10])

# Print the first 5 vectors of the tfidf training data
print(tfidf_train_deepcut.A[:5])

['1ดาว', '30ml', '50ml', '5ดาว', 'advance', 'advance night', 'advanced', 'age', 'air', 'amazing']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


attacut

In [ ]:
# Initialize a TfidfVectorizer object: tfidf_vectorizer
tfidf_vectorizer_attacut = TfidfVectorizer(analyzer=split_word_attacut, max_df=0.7)

# Transform the training data: tfidf_train 
tfidf_train_attacut = tfidf_vectorizer_attacut.fit_transform(X_train.values)

# Transform the test data: tfidf_test 
tfidf_test_attacut = tfidf_vectorizer_attacut.transform(X_test.values)

# Print the first 10 features
print(tfidf_vectorizer_attacut.get_feature_names()[:10])

# Print the first 5 vectors of the tfidf training data
print(tfidf_train_attacut.A[:5])

['advance', 'advanced', 'age', 'air', 'amazing', 'and', 'anr', 'ant', 'app', 'april']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


**Inspecting the vectors**

newmm

In [ ]:
# Create the CountVectorizer DataFrame: count_df
count_df_newmm = pd.DataFrame(count_train_newmm.A, columns=count_vectorizer_newmm.get_feature_names())

# Create the TfidfVectorizer DataFrame: tfidf_df
tfidf_df_newmm = pd.DataFrame(tfidf_train_newmm.A, columns=tfidf_vectorizer_newmm.get_feature_names())

# Print the head of count_df
print(count_df_newmm.head())

# Print the head of tfidf_df
print(tfidf_df_newmm.head())

   advance  advanced  age  air  amazing  and  anr  ...  ไหม  ไหล  ไหว  ไอ  ๅๅๅ  ้  ์
0        0         0    0    0        0    0    0  ...    0    0    0   0    0  0  0
1        0         0    0    0        0    0    0  ...    0    0    0   0    0  0  0
2        0         0    0    0        0    0    0  ...    0    0    0   0    0  0  0
3        0         0    0    0        0    0    0  ...    0    0    0   0    0  0  0
4        0         0    0    0        0    0    0  ...    0    0    0   0    0  0  0

[5 rows x 1628 columns]
   advance  advanced  age  air  amazing  and  ...  ไหล  ไหว   ไอ  ๅๅๅ    ้    ์
0      0.0       0.0  0.0  0.0      0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
1      0.0       0.0  0.0  0.0      0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2      0.0       0.0  0.0  0.0      0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
3      0.0       0.0  0.0  0.0      0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
4      0.0       0.0  0.0  0.0      0.0  0.0  ...  0.0  0.0  0.0 

In [ ]:
# Calculate the difference in columns: difference
difference = set(tfidf_df_newmm.columns) - set(count_df_newmm.columns)
print(difference)

# Check whether the DataFrames are equal
print(count_df_newmm.equals(tfidf_df_newmm))

set()
False


deepcut

In [ ]:
# Create the CountVectorizer DataFrame: count_df
count_df_deepcut = pd.DataFrame(count_train_deepcut.A, columns=count_vectorizer_deepcut.get_feature_names())

# Create the TfidfVectorizer DataFrame: tfidf_df
tfidf_df_deepcut = pd.DataFrame(tfidf_train_deepcut.A, columns=tfidf_vectorizer_deepcut.get_feature_names())

# Print the head of count_df
print(count_df_deepcut.head())

# Print the head of tfidf_df
print(tfidf_df_deepcut.head())

   1ดาว  30ml  50ml  5ดาว  advance  ...  ไหม  ไหร่  ไหล  ไหว  ไอเท็มฮ็อตฮิต
0     0     0     0     0        0  ...    0     0    0    0              0
1     0     0     0     0        0  ...    0     0    0    0              0
2     0     0     0     0        0  ...    0     0    0    0              0
3     0     0     0     0        0  ...    0     0    0    0              0
4     0     0     0     0        0  ...    0     0    0    0              0

[5 rows x 1450 columns]
   1ดาว  30ml  50ml  5ดาว  advance  ...  ไหม  ไหร่  ไหล  ไหว  ไอเท็มฮ็อตฮิต
0   0.0   0.0   0.0   0.0      0.0  ...  0.0   0.0  0.0  0.0            0.0
1   0.0   0.0   0.0   0.0      0.0  ...  0.0   0.0  0.0  0.0            0.0
2   0.0   0.0   0.0   0.0      0.0  ...  0.0   0.0  0.0  0.0            0.0
3   0.0   0.0   0.0   0.0      0.0  ...  0.0   0.0  0.0  0.0            0.0
4   0.0   0.0   0.0   0.0      0.0  ...  0.0   0.0  0.0  0.0            0.0

[5 rows x 1449 columns]


In [ ]:
# Calculate the difference in columns: difference
difference = set(tfidf_df_deepcut.columns) - set(count_df_deepcut.columns)
print(difference)

# Check whether the DataFrames are equal
print(count_df_deepcut.equals(tfidf_df_deepcut))

set()
False


attacut

In [ ]:
# Create the CountVectorizer DataFrame: count_df
count_df_attacut = pd.DataFrame(count_train_attacut.A, columns=count_vectorizer_attacut.get_feature_names())

# Create the TfidfVectorizer DataFrame: tfidf_df
tfidf_df_attacut = pd.DataFrame(tfidf_train_attacut.A, columns=tfidf_vectorizer_attacut.get_feature_names())

# Print the head of count_df
print(count_df_attacut.head())

# Print the head of tfidf_df
print(tfidf_df_attacut.head())

   advance  advanced  age  air  amazing  and  anr  ...  ไหม  ไหล  ไหว  ไอ  ๅๅๅ  ้  ์
0        0         0    0    0        0    0    0  ...    0    0    0   0    0  0  0
1        0         0    0    0        0    0    0  ...    0    0    0   0    0  0  0
2        0         0    0    0        0    0    0  ...    0    0    0   0    0  0  0
3        0         0    0    0        0    0    0  ...    0    0    0   0    0  0  0
4        0         0    0    0        0    0    0  ...    0    0    0   0    0  0  0

[5 rows x 1628 columns]
   advance  advanced  age  air  amazing  and  ...  ไหล  ไหว   ไอ  ๅๅๅ    ้    ์
0      0.0       0.0  0.0  0.0      0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
1      0.0       0.0  0.0  0.0      0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2      0.0       0.0  0.0  0.0      0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
3      0.0       0.0  0.0  0.0      0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
4      0.0       0.0  0.0  0.0      0.0  0.0  ...  0.0  0.0  0.0 

In [ ]:
# Calculate the difference in columns: difference
difference = set(tfidf_df_attacut.columns) - set(count_df_attacut.columns)
print(difference)

# Check whether the DataFrames are equal
print(count_df_attacut.equals(tfidf_df_attacut))

set()
False


**Training and testing model with CountVectorizer**

In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn import decomposition, ensemble
import xgboost

In [ ]:

def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    pred = classifier.predict(feature_vector_valid)

    # Calculate the accuracy score: score
    score = metrics.accuracy_score(y_test, pred)
    print(score)

    report = metrics.classification_report(y_test, pred)
    print(report)

    # Calculate the confusion matrix: cm
    cm = metrics.confusion_matrix(y_test, pred, labels=['0', '1'])
    print(cm)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return pred

Naive Bayes

newmm

In [ ]:
# Naive Bayes on Count Vectors
train_model(naive_bayes.MultinomialNB(), count_train_newmm, y_train, count_test_newmm)

# Naive Bayes on Word Level TF IDF Vectors
train_model(naive_bayes.MultinomialNB(), tfidf_train_newmm, y_train, tfidf_test_newmm)

# # Naive Bayes on Ngram Level TF IDF Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
# print ("NB, N-Gram Vectors: ", accuracy)

# # Naive Bayes on Character Level TF IDF Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
# print ("NB, CharLevel Vectors: ", accuracy)

0.9769911504424779
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1108
           1       0.44      0.73      0.55        22

    accuracy                           0.98      1130
   macro avg       0.72      0.85      0.77      1130
weighted avg       0.98      0.98      0.98      1130

[[1088   20]
 [   6   16]]
0.9805309734513274
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1108
           1       0.00      0.00      0.00        22

    accuracy                           0.98      1130
   macro avg       0.49      0.50      0.50      1130
weighted avg       0.96      0.98      0.97      1130

[[1108    0]
 [  22    0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


deepcut

In [ ]:
# Naive Bayes on Count Vectors
train_model(naive_bayes.MultinomialNB(), count_train_deepcut, y_train, count_test_deepcut)

# Naive Bayes on Word Level TF IDF Vectors
train_model(naive_bayes.MultinomialNB(), tfidf_train_deepcut, y_train, tfidf_test_deepcut)

0.9761061946902655
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1108
           1       0.43      0.68      0.53        22

    accuracy                           0.98      1130
   macro avg       0.71      0.83      0.76      1130
weighted avg       0.98      0.98      0.98      1130

[[1088   20]
 [   7   15]]
0.9805309734513274
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1108
           1       0.00      0.00      0.00        22

    accuracy                           0.98      1130
   macro avg       0.49      0.50      0.50      1130
weighted avg       0.96      0.98      0.97      1130

[[1108    0]
 [  22    0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


attacut

In [ ]:
# Naive Bayes on Count Vectors
train_model(naive_bayes.MultinomialNB(), count_train_attacut, y_train, count_test_attacut)

# Naive Bayes on Word Level TF IDF Vectors
train_model(naive_bayes.MultinomialNB(), tfidf_train_attacut, y_train, tfidf_test_attacut)

0.9769911504424779


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1108
           1       0.44      0.73      0.55        22

    accuracy                           0.98      1130
   macro avg       0.72      0.85      0.77      1130
weighted avg       0.98      0.98      0.98      1130

[[1088   20]
 [   6   16]]
0.9805309734513274
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1108
           1       0.00      0.00      0.00        22

    accuracy                           0.98      1130
   macro avg       0.49      0.50      0.50      1130
weighted avg       0.96      0.98      0.97      1130

[[1108    0]
 [  22    0]]


Logistic Regression

newmm

In [ ]:
# Logistic Regression on Count Vectors
train_model(linear_model.LogisticRegression(), count_train_newmm, y_train, count_test_newmm)

# Logistic Regression on Word Level TF IDF Vectors
train_model(linear_model.LogisticRegression(), tfidf_train_newmm, y_train, tfidf_test_newmm)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9858407079646018
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1108
           1       0.67      0.55      0.60        22

    accuracy                           0.99      1130
   macro avg       0.83      0.77      0.80      1130
weighted avg       0.98      0.99      0.99      1130

[[1102    6]
 [  10   12]]
0.9823008849557522
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1108
           1       1.00      0.09      0.17        22

    accuracy                           0.98      1130
   macro avg       0.99      0.55      0.58      1130
weighted avg       0.98      0.98      0.98      1130

[[1108    0]
 [  20    2]]


deepcut

In [ ]:
# Logistic Regression on Count Vectors
train_model(linear_model.LogisticRegression(), count_train_deepcut, y_train, count_test_deepcut)

# Logistic Regression on Word Level TF IDF Vectors
train_model(linear_model.LogisticRegression(), tfidf_train_deepcut, y_train, tfidf_test_deepcut)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9814159292035398
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1108
           1       0.53      0.36      0.43        22

    accuracy                           0.98      1130
   macro avg       0.76      0.68      0.71      1130
weighted avg       0.98      0.98      0.98      1130

[[1101    7]
 [  14    8]]
0.9805309734513274
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1108
           1       0.50      0.05      0.08        22

    accuracy                           0.98      1130
   macro avg       0.74      0.52      0.54      1130
weighted avg       0.97      0.98      0.97      1130

[[1107    1]
 [  21    1]]


attacut

In [ ]:
# Logistic Regression on Count Vectors
train_model(linear_model.LogisticRegression(), count_train_attacut, y_train, count_test_attacut)

# Logistic Regression on Word Level TF IDF Vectors
train_model(linear_model.LogisticRegression(), tfidf_train_attacut, y_train, tfidf_test_attacut)

0.9858407079646018


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1108
           1       0.67      0.55      0.60        22

    accuracy                           0.99      1130
   macro avg       0.83      0.77      0.80      1130
weighted avg       0.98      0.99      0.99      1130

[[1102    6]
 [  10   12]]
0.9823008849557522
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1108
           1       1.00      0.09      0.17        22

    accuracy                           0.98      1130
   macro avg       0.99      0.55      0.58      1130
weighted avg       0.98      0.98      0.98      1130

[[1108    0]
 [  20    2]]


Random Forest

newmm

In [ ]:
# Random Forest on Count Vectors
train_model(ensemble.RandomForestClassifier(), count_train_newmm, y_train, count_test_newmm)

# Random Forest on Word Level TF IDF Vectors
train_model(ensemble.RandomForestClassifier(), tfidf_train_newmm, y_train, tfidf_test_newmm)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9831858407079646
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1108
           1       0.71      0.23      0.34        22

    accuracy                           0.98      1130
   macro avg       0.85      0.61      0.67      1130
weighted avg       0.98      0.98      0.98      1130

[[1106    2]
 [  17    5]]


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9814159292035398
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1108
           1       0.57      0.18      0.28        22

    accuracy                           0.98      1130
   macro avg       0.78      0.59      0.63      1130
weighted avg       0.98      0.98      0.98      1130

[[1105    3]
 [  18    4]]


deepcut

In [ ]:
# Random Forest on Count Vectors
train_model(ensemble.RandomForestClassifier(), count_train_deepcut, y_train, count_test_deepcut)

# Random Forest on Word Level TF IDF Vectors
train_model(ensemble.RandomForestClassifier(), tfidf_train_deepcut, y_train, tfidf_test_deepcut)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9858407079646018
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1108
           1       0.88      0.32      0.47        22

    accuracy                           0.99      1130
   macro avg       0.93      0.66      0.73      1130
weighted avg       0.98      0.99      0.98      1130

[[1107    1]
 [  15    7]]


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9831858407079646
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1108
           1       0.71      0.23      0.34        22

    accuracy                           0.98      1130
   macro avg       0.85      0.61      0.67      1130
weighted avg       0.98      0.98      0.98      1130

[[1106    2]
 [  17    5]]


attacut

In [ ]:
# Random Forest on Count Vectors
train_model(ensemble.RandomForestClassifier(), count_train_attacut, y_train, count_test_attacut)

# Random Forest on Word Level TF IDF Vectors
train_model(ensemble.RandomForestClassifier(), tfidf_train_attacut, y_train, tfidf_test_attacut)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9814159292035398
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1108
           1       0.57      0.18      0.28        22

    accuracy                           0.98      1130
   macro avg       0.78      0.59      0.63      1130
weighted avg       0.98      0.98      0.98      1130

[[1105    3]
 [  18    4]]


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9823008849557522
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1108
           1       0.67      0.18      0.29        22

    accuracy                           0.98      1130
   macro avg       0.83      0.59      0.64      1130
weighted avg       0.98      0.98      0.98      1130

[[1106    2]
 [  18    4]]


Boosting Model

newmm

In [ ]:
# Extereme Gradient Boosting on Count Vectors
train_model(xgboost.XGBClassifier(), count_train_newmm.tocsc(), y_train, count_test_newmm.tocsc())

# Extereme Gradient Boosting on Word Level TF IDF Vectors
train_model(xgboost.XGBClassifier(), tfidf_train_newmm.tocsc(), y_train, tfidf_test_newmm.tocsc())

0.9823008849557522
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1108
           1       0.60      0.27      0.37        22

    accuracy                           0.98      1130
   macro avg       0.79      0.63      0.68      1130
weighted avg       0.98      0.98      0.98      1130

[[1104    4]
 [  16    6]]
0.9814159292035398
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1108
           1       0.56      0.23      0.32        22

    accuracy                           0.98      1130
   macro avg       0.77      0.61      0.66      1130
weighted avg       0.98      0.98      0.98      1130

[[1104    4]
 [  17    5]]


deepcut

In [ ]:
# Extereme Gradient Boosting on Count Vectors
train_model(xgboost.XGBClassifier(), count_train_deepcut.tocsc(), y_train, count_test_deepcut.tocsc())

# Extereme Gradient Boosting on Word Level TF IDF Vectors
train_model(xgboost.XGBClassifier(), tfidf_train_deepcut.tocsc(), y_train, tfidf_test_deepcut.tocsc())

0.9814159292035398
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1108
           1       0.57      0.18      0.28        22

    accuracy                           0.98      1130
   macro avg       0.78      0.59      0.63      1130
weighted avg       0.98      0.98      0.98      1130

[[1105    3]
 [  18    4]]
0.9823008849557522
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1108
           1       0.62      0.23      0.33        22

    accuracy                           0.98      1130
   macro avg       0.80      0.61      0.66      1130
weighted avg       0.98      0.98      0.98      1130

[[1105    3]
 [  17    5]]


attacut

In [ ]:
# Extereme Gradient Boosting on Count Vectors
train_model(xgboost.XGBClassifier(), count_train_attacut.tocsc(), y_train, count_test_attacut.tocsc())

# Extereme Gradient Boosting on Word Level TF IDF Vectors
train_model(xgboost.XGBClassifier(), tfidf_train_attacut.tocsc(), y_train, tfidf_test_attacut.tocsc())

0.9823008849557522
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1108
           1       0.60      0.27      0.37        22

    accuracy                           0.98      1130
   macro avg       0.79      0.63      0.68      1130
weighted avg       0.98      0.98      0.98      1130

[[1104    4]
 [  16    6]]
0.9814159292035398
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1108
           1       0.56      0.23      0.32        22

    accuracy                           0.98      1130
   macro avg       0.77      0.61      0.66      1130
weighted avg       0.98      0.98      0.98      1130

[[1104    4]
 [  17    5]]


In [ ]:
pred = train_model(linear_model.LogisticRegression(), count_train_newmm, y_train, count_test_newmm)
pred_df = pd.DataFrame(pred, columns=['pred'])
pred_test_data = pd.concat([test_data, pred_df],axis=1)

0.9858407079646018
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1108
           1       0.67      0.55      0.60        22

    accuracy                           0.99      1130
   macro avg       0.83      0.77      0.80      1130
weighted avg       0.98      0.99      0.99      1130

[[1102    6]
 [  10   12]]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [ ]:
for shopid in pred_test_data.shopid.unique():
  #print(shopid)
  df = pred_test_data[pred_test_data['shopid'] == shopid]
  pred_count = df.pred.value_counts()
  actual_count = df.label.value_counts()
  print('shopid: ',shopid,',0:', pred_count[0],',1:',pred_count[1])
  print('predict propability: ',round((pred_count[1]+1)/(pred_count[0]+pred_count[1]+2), 2) )
  print('actual propability: ',round((actual_count[1]+1)/(actual_count[0]+actual_count[1]+2), 2),'\n' )


  #edit wrong typing

shopid:  4343306 ,0: 254 ,1: 1
predict propability:  0.01
actual propability:  0.02 

shopid:  1514953 ,0: 326 ,1: 1
predict propability:  0.01
actual propability:  0.01 

shopid:  773245 ,0: 224 ,1: 3
predict propability:  0.02
actual propability:  0.01 

shopid:  50796065 ,0: 301 ,1: 9
predict propability:  0.03
actual propability:  0.03 

shopid:  83258020 ,0: 7 ,1: 4
predict propability:  0.38
actual propability:  0.46 



In [ ]:
def show_classfeats(df, vectorizer, analyzer, score_name='', text_col='texts', class_col='category', is_emoji=False):
    vec = vectorizer(analyzer=analyzer)
    mat = vec.fit_transform(df[text_col])
    dfs = top_feats_by_class(mat, df[class_col], vec.get_feature_names())

    if is_emoji:
        for dataframe in dfs:
            dataframe.columns = [dataframe.label, 'tfidf_' + dataframe.label]
        return pd.concat(dfs, axis=1)
    else:
        #plot_classfeats_h(dfs, score_name=score_name)
        print(dfs)

In [ ]:
from matplotlib import pyplot as plt
import string
import emoji
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
import matplotlib
import matplotlib as mpl
import matplotlib.font_manager as fm

!wget https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf
!cp thsarabunnew-webfont.ttf /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/fonts/ttf/
!cp thsarabunnew-webfont.ttf /usr/share/fonts/truetype/

--2019-11-14 07:50:30--  https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Phonbopit/sarabun-webfont/master/fonts/thsarabunnew-webfont.ttf [following]
--2019-11-14 07:50:31--  https://raw.githubusercontent.com/Phonbopit/sarabun-webfont/master/fonts/thsarabunnew-webfont.ttf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98308 (96K) [application/octet-stream]
Saving to: ‘thsarabunnew-webfont.ttf’

thsarabunnew-webfon 100%[===================>]  96.00K  --.-KB/s    in 0.04s   

2019-11-14 07:50:32 (2.48 MB/s) - ‘th

In [ ]:
matplotlib.font_manager._rebuild()
matplotlib.rc('font', family='TH Sarabun New')

In [ ]:
def plot_classfeats_h(dfs, score_name=''):
    ''' Plot the data frames returned by the function plot_tfidf_classfeats(). '''

    fig = plt.figure(figsize=(12, 9), facecolor="w")
    x = np.arange(len(dfs[0]))

    for i, df in enumerate(dfs):
        ax = fig.add_subplot(1, len(dfs), i+1)
        # ax.spines["top"].set_visible(False)
        # ax.spines["right"].set_visible(False)
        # ax.set_frame_on(False)
        # ax.get_xaxis().tick_bottom()
        # ax.get_yaxis().tick_left()
        ax.set_xlabel(f"Mean {score_name} Score", labelpad=16, fontsize=14)
        ax.set_title("label = " + str(df.label), fontsize=16)
        ax.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
        ax.barh(x, df.score, align='center', color='#3F5D7D')
        ax.set_yticks(x)
        ax.set_ylim([-1, x[-1]+1])
        ax.invert_yaxis()
        yticks = ax.set_yticklabels(df.feature)
        plt.subplots_adjust(bottom=0.09, right=0.97, left=0.15, top=0.95, wspace=0.52)

    plt.show()

def top_feats_by_class(Xtr, y, features, min_tfidf=0.1, top_n=25):
    ''' Return a list of dfs, where each df holds top_n features and their mean tfidf value
        calculated across documents with the same class label. '''

    dfs = []
    labels = np.unique(y)

    for label in labels:
        ids = np.where(y==label)
        feats_df = top_mean_feats(Xtr, features, ids, min_tfidf=min_tfidf, top_n=top_n)
        feats_df.label = label
        dfs.append(feats_df)

    return dfs

def top_mean_feats(Xtr, features, grp_ids=None, min_tfidf=0.1, top_n=25):
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in grp_ids. '''

    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)

    return top_feats(tfidf_means, features, top_n)

def top_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''

    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ["feature", "score"]

    return df

In [ ]:
def show_classfeats(df, vectorizer, analyzer, score_name='', text_col='texts', class_col='category', is_emoji=False):
    vec = vectorizer(analyzer=analyzer)
    mat = vec.fit_transform(df[text_col])
    dfs = top_feats_by_class(mat, df[class_col], vec.get_feature_names())

    if is_emoji:
        for dataframe in dfs:
            dataframe.columns = [dataframe.label, 'tfidf_' + dataframe.label]
        return pd.concat(dfs, axis=1)
    else:
        #plot_classfeats_h(dfs, score_name=score_name)
        print(dfs)

In [ ]:
def process_text(text):
    tokens = split_word_newmm(text) 
    # stop = tuple(thai_stopwords())
    # tokens = word_tokenize(text, engine='attacut', keep_whitespace=False)
    
    # Remove stop words ภาษาไทย และภาษาอังกฤษ
    # tokens = [i for i in tokens if not i in th_stop and not i in en_stop]

    # nopunc = [char for char in text if char not in string.punctuation]
    # nopunc = "".join(nopunc)
    
    return tokens

In [ ]:
# newmm with custom dict
show_classfeats(
    df=train_data,
    vectorizer=TfidfVectorizer,
    analyzer=process_text,
    score_name="tf-idf",
    text_col='text',
    class_col='label'
)

[        feature     score
0         ดีมาก  0.192835
1        สินค้า  0.097413
2        คุณภาพ  0.084594
3     การจัดส่ง  0.080464
4            ดี  0.076515
5   ความคุ้มค่า  0.070219
6           ส่ง  0.054677
7            กก  0.051286
8            ไว  0.039767
9     ให้บริการ  0.038172
10       ของแท้  0.033707
11         สั่ง  0.028724
12      ร้านค้า  0.027748
13         ราคา  0.027622
14         ร้าน  0.025964
15       จัดส่ง  0.024829
16         ซื้อ  0.023664
17          ห่อ  0.023058
18         แพ็ค  0.023048
19       แน่นอน  0.022216
20           แพ  0.019840
21        ขนส่ง  0.019673
22            ค  0.019195
23          ขวด  0.018760
24         นะคะ  0.018221,       feature     score
0       เนื้อ  0.112539
1       กลิ่น  0.111013
2        เหลว  0.093340
3      เหมือน  0.062123
4        แปลก  0.060764
5         ขวด  0.059770
6        รั่ม  0.057573
7          เซ  0.055627
8          สี  0.046749
9   เหมือนกัน  0.037858
10       ปลอม  0.037294
11      ไม่มี  0.033289
12     แย่

In [ ]:
# deepcut
show_classfeats(
    df=train_df,
    vectorizer=TfidfVectorizer,
    analyzer=process_text,
    score_name="tf-idf",
    text_col='text',
    class_col='label'
)

[    feature     score
0       แย่  0.113201
1        ดี  0.090645
2    สินค้า  0.065791
3     กล่อง  0.058257
4     พอใช้  0.044411
5    บริการ  0.042601
6       ขวด  0.039956
7       ห่อ  0.038048
8      ร้าน  0.037076
9       ค้า  0.034071
10   คุณภาพ  0.032608
11  คุ้มค่า  0.032282
12     เก่า  0.028157
13     แพ็ค  0.024292
14     สั่ง  0.024005
15       ไว  0.022791
16      ตอบ  0.021842
17     โอเค  0.019120
18      แชท  0.018931
19    ทดลอง  0.018918
20  ขีดข่วน  0.017123
21      รอย  0.016933
22     เช็ค  0.016683
23      บุบ  0.016236
24     2017  0.016193,    feature     score
0    กลิ่น  0.092257
1   เหมือน  0.078598
2      ขวด  0.076909
3    เนื้อ  0.066800
4   สินค้า  0.059738
5     ซื้อ  0.051740
6       ดู  0.046287
7     แปลก  0.041672
8      รอย  0.040543
9    แน่ใจ  0.034675
10    เหลว  0.034526
11      สี  0.034498
12      ดี  0.034336
13    ปลอม  0.032851
14  คุณภาพ  0.031028
15    ปกติ  0.030462
16     ตัว  0.029907
17  เซรั่ม  0.028638
18     จาง  0.023420
19    

In [ ]:
# attacut
show_classfeats(
    df=train_df,
    vectorizer=TfidfVectorizer,
    analyzer=process_text,
    score_name="tf-idf",
    text_col='text',
    class_col='label'
)

[        feature     score
0     การจัดส่ง  0.081370
1         พอใช้  0.062821
2           แย่  0.061858
3            ดี  0.058642
4        สินค้า  0.057104
5         กล่อง  0.054452
6        แย่มาก  0.041082
7           ขวด  0.040676
8           ห่อ  0.036990
9        คุณภาพ  0.032263
10        ดีมาก  0.030455
11         แพ็ค  0.030402
12      ร้านค้า  0.030189
13    ให้บริการ  0.028891
14  ความคุ้มค่า  0.028814
15         เก่า  0.028385
16        ไม่มี  0.027937
17         โอเค  0.027226
18   รอยขีดข่วน  0.025957
19           ไว  0.023370
20          แชท  0.021856
21          ตอบ  0.021641
22         นะคะ  0.019172
23           แพ  0.017531
24            ค  0.017531,       feature     score
0       กลิ่น  0.079389
1         ขวด  0.074387
2      เหมือน  0.063626
3       เนื้อ  0.060660
4      ของแท้  0.050591
5      สินค้า  0.049196
6        ซื้อ  0.043602
7          เซ  0.040433
8        รั่ม  0.039284
9        เหลว  0.038385
10       แปลก  0.037903
11         ดู  0.033589
12   ไม่แน